In [ ]:
# initial steps
python3.11 -m pip install pandas dash

wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

python3.11 spacex_dash_app.py


# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe. max/min_payloads needed for slider
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()         
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                                    ],
                                            value='ALL',
                                            placeholder="Select a launch site here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(min=0, max=10000, step=1000, 
                                                value=[min_payload, max_payload], 
                                                marks={i: str(i) for i in range(0,10001,2500)},
                                                id='payload-slider'),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, 
                    values='class', 
                    names='Launch Site', 
                    title='Total Successful Launches by Site'
                    )
        return fig
    else:
        # return the outcomes piechart for a selected site

        # filters for selected launch site
        filtered_df = filtered_df[filtered_df['Launch Site']==entered_site]

        # create new df with counts of 0s and 1s, then reset index to default names ['class', 'count']
        class_counts = filtered_df['class'].value_counts().reset_index()

        # create the pie chart using new df 
        fig = px.pie(class_counts,
                    names = 'class',
                    values = 'count',
                    title = 'Total Successful Launches for Site ' + str(entered_site)
                    )
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value")]
             )

def get_scatter_chart(entered_site, payload_range):
    
    # split list payload_range from slider into two separate values for filtering df
    min_payload, max_payload = payload_range  
    
    # df filters for payload mass in slider range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & (spacex_df['Payload Mass (kg)'] <= max_payload)]

    if entered_site == 'ALL':
        fig = px.scatter(
                    filtered_df,
                    x = 'Payload Mass (kg)',
                    y = 'class',
                    color = "Booster Version Category",
                    title = 'Correlation between Payload and Success for all Sites'
                    )

    else: 
        filtered_df = filtered_df[filtered_df['Launch Site']==entered_site]
    
        fig = px.scatter(
                    filtered_df,
                    x = 'Payload Mass (kg)',
                    y = 'class',
                    color = "Booster Version Category",
                    title = 'Correlation between Payload and Success for Site ' +str(entered_site)
                    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()